# Managing an HPE SDF 280 RMC with Redfish

In [ ]:
########## Environment preparation (Version: 0.30) ############


# Set Student ID number
export stdid={{ STDID }}
Id=$(id --user --name)

# location and ports variables
ObmcBasePort={{ OBMCBASEPORT }}
iLO5DlBasePort=45000
iLO5SyBasePort=46000
RMCSdfBasePort=47000

let OpenBmcPort=${ObmcBasePort}+${stdid}
let iLO5DlSimulatorPort=$iLO5DlBasePort+${stdid}
let iLO5SySimulatorPort=${iLO5SyBasePort}+${stdid}
let RMCSdfSimulatorPort=${RMCSdfBasePort}+${stdid}
let ilo5Port=443

CacheDlDir="${PWD}/iLOrestCacheDl"
CacheSyDir="${PWD}/iLOrestCacheSy"
CacheSdfDir="${PWD}/iLOrestCacheSdf"
LogDir="${PWD}"
LogFile="${LogDir}/iLOrest.log"

iLO5DlSimulatorIP=ilo5simulators
iLO5SySimulatorIP=ilo5simulators
RMCSdfSimulatorIP=ilo5simulators
OpenBmcIP=openbmcsimulators

iLO5DlSimulator=${iLO5SimulatorIP}:${iLO5DlSimulatorPort}
iLO5DlSimulatorURI=https://${iLO5DlSimulator}

iLO5SySimulator=${iLO5SimulatorIP}:${iLO5SySimulatorPort}
iLO5SySimulatorURI=https://${iLO5SySimulator}

RMCSdfSimulator=${RMCSdfSimulatorIP}:${RMCSdfSimulatorPort}
RMCSdfSimulatorURI=https://${RMCSdfSimulator}

OpenBmc="${OpenBmcIP}:${OpenBmcPort}"
OpenBmcURI="https://${OpenBmc}"

ilo5IP="ilo5"
ilo5="${ilo5IP}:${ilo5Port}"
ilo5URI="https://${ilo5}"

# Credentials
iLOUser="student"
iLOPassword={{ REDFISHPWD }}
RmcUser="administrator"
RmcPassword={{ REDFISHPWD }}

# Miscellaneous
WorkshopDir=$PWD
HpePythonRedfishVenv="HpePythonRedfishVenv"
#export PYTHONPATH="${WorkshopDir}/${NbId}/library/"
w=$(basename $PWD)

alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null ; sleep 1"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in OpenBmc ilo5 iLO5DlSimulator iLO5SySimulator RMCSdfSimulator ; do
    ip="${bmc}IP" ; port="${bmc}Port"
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null &&
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

echo
# Retrieve iLO firmware versions from ServiceRoot (no credentials needed)
for bmc in ilo5 iLO5DlSimulator iLO5SySimulator ; do
    ip="${bmc}IP" ; port="${bmc}Port"
    echo -n "$bmc firmware version: "
    curl --silent --insecure -X GET https://$(eval echo \$${ip}):$(eval echo \$${port})/redfish/v1 | \
         jq  '[.Oem.Hpe.Manager[]] | .[] | .ManagerFirmwareVersion' 2>/dev/null
done

echo 
# Print ilorest version
ilorest --version

## Restart simulators

In [ ]:
# Simulators restart
ResetSimulators

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5DlSimulator iLO5SySimulator RMCSdfSimulator ; do
    ip="${bmc}IP" ; port="${bmc}Port"
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

The following command dumps the RMC top-level model. 

In [ ]:
# Get RMC information
curl --insecure --silent --user ${RmcUser}:${RmcPassword} \
     --request GET \
     ${RMCSdfSimulatorURI}/redfish/v1/Managers/RMC | python3 -m json.tool


You can continue to run individual curl commands, each providing the username/
password or or you can create a session

## Sessions

Redfish supports a facility to retrieve an authentication token to use for a session, in the same way a browser session does for https. To get a
token for a session, POST to /redfish/v1/SessionService/Sessions/ to create a session. The authentication token is part of the
response from that POST. Use the authentication token in subsequent commands, then (as a best practice), remove your session from
SessionService when you are done. Complete details are included in how-to videos on redfish.dmtf.org/.

The following combines the POST with a variable set, to grab the authentication token for later use.


In [ ]:
curl --silent --include --insecure \
     --request POST \
     --header 'Content-Type: application/json' \
     --data '{"UserName":"'${RmsUser}'","Password":"'${RmcPassword}'"}' \
     ${RMCSdfSimulatorURI}/redfish/v1/SessionService/Sessions/ # | awk '/X-Auth-Token/ {print $2} ' | tr -d '\r\n'

export AUTH_TOKEN="FakeToken"

## Power on

In [ ]:
curl --silent --insecure --include \
     --header "X-Auth-Token: ${AUTH_TOKEN}" \
     --header 'Content-Type: application/json' \
     -X POST  -d '{"ResetType": "On"}' \
     ${RMCSdfSimulatorURI}/redfish/v1/Systems/Partition0/Actions/ComputerSystem.Reset

In [ ]:
## Delete session

In [ ]:
curl --include --insecure \
     -H "X-Auth-Token: ${AUTH_TOKEN}" \
     -H 'Content-Type: application/json'
     -X DELETE  \
      ${RMCSdfSimulatorURI}/redfish/v1/SessionService/Sessions/administrator0c6f70f1959f47718e70dfda02d4ef4a

## DMTF Redfishtool

In [ ]:
# Create Virtual Python environment (Venv)
[ -d ${HpePythonRedfishVenv} ] && rm -rf ${HpePythonRedfishVenv} &>/dev/null 
python3 -m venv ${HpePythonRedfishVenv}                         &>/dev/null
source ${HpePythonRedfishVenv}/bin/activate                     &>/dev/null
PS1="[PEXP\[\]ECT_PROMPT>"                                      # Avoid Venv long prompt messing up outputs

# Install latest Ansible in the Venv
pip install redfishtool                                         &>/dev/null

echo -e "\n\n\tFinished creating Venv\n\n"

In [ ]:

redfishtool.py --Secure=Always --rhost=$RMCSdfSimulator -u $RmcUser -p $RmcPassword serviceRoot


## List partitions

In [ ]:
redfishtool.py --rhost=$RMCSdfSimulator --Secure=Always  -u $RmcUser -p $RmcPassword Systems

## Top level resources for Partition 0

In [ ]:
redfishtool.py --rhost=$RMCSdfSimulator --Secure=Always  -u $RmcUser -p $RmcPassword Systems -I Partition0

In [ ]:
redfishtool.py --rhost=$RMCSdfSimulator --Secure=Always  -u $RmcUser -p $RmcPassword Systems -I Partition0 -P Status

## DMTF Tackle box

In [ ]:
pip install redfish_utilities

In [ ]:
echo $RMCSdfSimulator
rf_sensor_list.py -u $RmcUser -p $RmcPassword -r https://$RMCSdfSimulator